In [0]:
scaling_factor = 0.1068 #microns per pixel, 100x objective

# parameters used for adaptive threshoding of the nucleus in the RFP channel, see below
bloc_size_frac_to_use = 0.02
offset_to_use = -100



initial_table_recalculated = pd.DataFrame({})

PATH_to_the_TIFs = #folder containing the tiff images

c = 0

# every position imaged on the microfluidic chip was saved as a separated tiff file
for pos in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
  
  
    #loading tiff files containing all the images for one position

    filename = os.path.join(skimage.data_dir, PATH_to_the_TIFs+#name of the file#+pos)
    cell_tif = io.imread(filename)
    
    
    plt.ioff()

    
    # the variable 'individual_cells' contains the name of every cells segmented for every position, ex: pos01_1,pos01_2,...,pos02_1
    for cell in individual_cells:
        if 'pos'+pos in cell:
          
                # loading the table containing the BudJ parameters for one cell: x, y, major_R,minor_r, angle
                temp = initial_table[initial_table["Cell_pos"] == cell]
                temp = temp.sort_values(by="TimeID")
                time_axis = list(temp["TimeID"])
                
                 
                RFP_mean = []
                GFP_nuc_mean = []
                GFP_cyto_mean = []
                Nuc_size = []
                GFP_total_mean = []
                    
                for t in time_axis:
                   
                    t_in_tiff = int(t)
                    
                    h, k, a, b, A = ellipse(t, temp, scaling_factor)   #function that converts the parameters of the ellipse at time point t (given by BudJ) from micron to pixel
                    # h=x, k=y, a=major R, b=minor r, A=angle 
                    
                    
                    image = cell_tif[t_in_tiff-1,:,:,:,2] #does not matter which channel
                    nrows =image.shape[1]   #getting the size of the image
                    ncols =image.shape[2]
                    row, col = np.ogrid[:nrows, :ncols]
                    
                    
                    # creting a mask with only the pixels within the segmented ellipse at time point t
                    inner_disk_mask = ((((col-h)*math.cos(A)+(row-k)*math.sin(A))**2)/(a**2) + (((col-h)*math.sin(A)-(row-k)*math.cos(A))**2)/(b**2) - 1 < 0)
                                        

                    RFP_nucleus = []
                    GFP_nucleus = []
                    GFP_cyto = []
                    GFP_total = []
                    size = []
                    
                    
                    # for every z-stack we will use the RFP image to calculate a mask for the nucleus and use that mask to calculate the GFP intensity in the nucleus and in the cytoplasm
                    for z in range(0, 5):
                        
                        image = cell_tif[t_in_tiff-1,z,:,:,1]    #loading RFP image for stack z and time t_in_tiff-1
                        image_masked_flattened = image[inner_disk_mask == True]
                        imageGFP = cell_tif[t_in_tiff-1,z,:,:,2]#/fitted_curve1[t_in_tiff-1]    #photobleaching correction, to be calculated separately with exponential fitting of the overall intesity over time
                        
                        image = image*inner_disk_mask
                        num_cell_pixels = len(image_masked_flattened)
                        
                        
                        # calculating a mask of the nucleus by adaptive thresholding and image cleaning, this way only the brightest pixels of the RFP image, hence the inner part of the nucleus,
                        # will be used as a nuclear mask
                    
                        bloc_size_cell_size_dependent = round_up_to_odd(bloc_size_frac_to_use*num_cell_pixels)    #adaptove thresholding will be done only on a part of the whole pixels inside the ellipse mask
                        ad_thr_mask = threshold_adaptive(image, bloc_size_cell_size_dependent, offset=offset_to_use)
                        ad_thr_mask1 = remove_small_objects(ad_thr_mask, min_size=10)   #removing small obljects to clean up the mask
                        ad_thr_mask2 = scipy.ndimage.morphology.binary_fill_holes(ad_thr_mask1)  #filling, dilation and erosion of the mask
                        ad_thr_mask3 = scipy.ndimage.morphology.binary_dilation(ad_thr_mask2)
                        ad_thr_mask4 = scipy.ndimage.morphology.binary_erosion(ad_thr_mask3, structure = np.ones((3,3))) ####final mask of the nucleus  
                        
                        
                        
                        
                        # a bigger mask of the nucleus will be calculated by a simply intesity threshold plus some image cleaning in order to 
                        # exclude from the whole cell mask the whole nucleus
                        image_tresh2 = image > 20
                        
                        nuc_mask = remove_small_objects(image_tresh2, min_size=15)
                        nuc_mask1 = scipy.ndimage.morphology.binary_fill_holes(nuc_mask)
                        nuc_mask2 = scipy.ndimage.morphology.binary_dilation(nuc_mask1)
                        nuc_mask3 = scipy.ndimage.morphology.binary_erosion(nuc_mask2, structure = np.ones((3,3)))
                        
                        cyto_mask = inner_disk_mask - nuc_mask3 
                        
                     
                        
                        image_cyto = imageGFP*cyto_mask
                        
                        # all the pixels with an intensity lower than the fifty percentile will be exclude from the cytosolic mask in order to exclude the vacuoles
                        up = np.percentile(image_cyto[cyto_mask==True],50)
                        image_cyto_corr = image_cyto > up
                        
      
                        ad_thr_mask2 = remove_small_objects(image_cyto_corr, min_size=10) ####final mask of the cytosol
                        
                       
                        imageRFP = cell_tif[t_in_tiff-1,z,:,:,1]
                        
                        
                        # calculating the mean intensity of the GFP and RFP channel inside and outside of the nucleus    
                        size_nuc = image[ad_thr_mask4 == True]
                        mean_rfp = np.mean(imageRFP[ad_thr_mask4 == True])
                        mean_gfp_nuc = np.mean(imageGFP[ad_thr_mask4 == True])
                        mean_gfp_cyto = np.mean(imageGFP[ad_thr_mask2 == True])
                        mean_gfp_total = np.mean(imageGFP[inner_disk_mask == True])
                        RFP_nucleus.append(mean_rfp)
                        GFP_nucleus.append(mean_gfp_nuc)
                        GFP_cyto.append(mean_gfp_cyto)
                        size.append(len(size_nuc))
                        GFP_total.append(mean_gfp_total)
                     
                    #removing 'nan' values
                    RFP_nucleus[RFP_nucleus == 'nan'] = 0
                    GFP_nucleus[GFP_nucleus == 'nan'] = 0
                    GFP_cyto[GFP_cyto == 'nan'] = 0
                    GFP_total[GFP_total == 'nan'] = 0
                    size[size == 'nan'] = 0    
                    
                    
                    #from all the z-stacks the one with the highest mean intesity (in the GFP or RFP channel) will be chosen as the most correct value
                    RFP_mean.append(max(RFP_nucleus))
                    GFP_nuc_mean.append(max(GFP_nucleus))
                    GFP_cyto_mean.append(max(GFP_cyto))
                    GFP_total_mean.append(max(GFP_total))
                        
                    Nuc_size.append(max(size))
                    
                    
                    
                temp["RFP_mean"] = pd.Series(RFP_mean, index=temp.index)
                temp["GFP_nuc_mean"] = pd.Series(GFP_nuc_mean, index=temp.index)
                temp["GFP_cyto_mean"] = pd.Series(GFP_cyto_mean, index=temp.index)
                temp["GFP_total_mean"] = pd.Series(GFP_total_mean, index=temp.index)
                temp["Nuclear_volume"] = pd.Series(Nuc_size, index=temp.index)
                    
                    
                initial_table_recalculated = pd.concat([initial_table_recalculated, temp])  
                

                
                
                c += 1
                print cell,